Comparing predicted and actual CDFs
---------

Setting up the scenario:

You have a **probabilistic forecast prediction**. In this case, you are modeling the uncertainty in the prediction using a Gaussian distribution, meaning the mean of the Gaussian is your prediction and the sigma is the 1-sigma uncertainty on your prediction.

The prediction values are simply taken from a random number generator: the mean is uniformly distributed between 0 and 1, and the sigma is uniformly distributed between 0.8 and 1.2.

In this scenario, the prediction is a pretty good one, meaning that the true values are indeed close to the mean that you predict. However, your prediction uncertainty is too small; the true values actually have a bigger spread than you predicted, by about 20%. This scenario is set up below:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.interpolate import interp1d

In [ ]:
predicted_mean = np.random.uniform(size=(5000))
predicted_sigma = np.random.uniform(low=0.8,high=1.2,size=(5000))

noise = np.random.normal(0.0,0.02,size=(5000))
actualDist = np.random.normal(predicted_mean,predicted_sigma*1.2,size=(5000)) + noise

Plot true versus predicted CDF
--------

In order to visualize the performance of the probabilistic forecast, we want to check whether the location/spread of true values matches the spread that we predicted. At each timestep, there exists a PDF of the prediction as well as the true value (the single value corresponding to what actually happened). This means that at each timestep, you can calculate the quantile (or percentile) at which the true value fell. This value corresponds to the "predicted CDF value" (a float between 0 and 1) of the true value, i.e. where in the CDF the true value landed.

In [ ]:
def _predicted_CDF(true_value,mean,sigma) :
    return stats.norm.cdf(true_value,mean,sigma)

predictedCDF = np.vectorize(_predicted_CDF)(actualDist,predicted_mean,predicted_sigma)

Accumulating these predicted CDF values allows one to check whether the CDF modeling (and therefore the PDF modeling) was accurate in the aggregate. In other words, 25% of the true values should fall within each quantile of your prediction. If not, then your PDF is somehow not capturing the mean or uncertainty correctly.

We therefore create a CDF of the predicted CDF values (using `cumsum`), which will tell us the **actual** distribution of the true events compared to the expectation. Then we plot the actual vs the predicted CDFs.

If the predictions and their uncertainty have good "coverage", then this line will be diagonal between (0,0) and (1,1), which means that things we predicted to happen 1% of the time actually happened 1% of the time ... things that we predicted to happen 5% of the time actually happened 5% of the time... etc.

If the prediction uncertainties are too small, then the line will be above the diagonal on the left and below the diagonal on the right (as seen in the plot). If prediction uncertainties are too big, then the line will be below (above) the diagonal on the left (right). If the line does not go through the point (0.5,0.5), then this is a sign that your median prediction is biased below or above the true value.

In [ ]:
predictedCDF_hist,bins = np.histogram(predictedCDF,1000,range=(0,1))
predictedCDF_hist_rough,bins_rough = np.histogram(predictedCDF,100,range=(0,1))

predictedCDF_values = bins[1:]
actualCDF_values = np.cumsum(predictedCDF_hist)/sum(predictedCDF_hist)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[5,5])
ax.stairs(predictedCDF_hist_rough, bins_rough)
ax.text(0.15,0.75*(ax.get_ylim()[1]-ax.get_ylim()[0])+ax.get_ylim()[0],'For a perfectly calibrated forecast,\nthis is a flat line.')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[5,5])
ax.plot(predictedCDF_values, actualCDF_values, label='true vs predicted cdf',linestyle='--')
ax.plot([0,1],[0,1],label='perfect coverage')
ax.plot([0.5],[0.5],label='Median is accurate')

ax.set_xticks(np.linspace(0,1,21),minor=True)
ax.set_yticks(np.linspace(0,1,21),minor=True)
ax.set(xlabel='Percentile assuming X distribution',ylabel='Actual percentile in data')
ax.grid(which='both')
ax.legend()

To find a certain value along this line, we use interpolation to probe this line. In the example below, we check on events in which the true value lay **below the $-2\sigma$ threshold** (i.e. should occur less than 5% of the time). We find that this type of event actually occurs 8.9% of the time. 

In [ ]:
def interpolate_predicted_to_actual(actual_cdf,predicted_cdf,percent,invert=False) :
    # use an interpolator to create a ppf
    interp = interp1d(predicted_cdf, actual_cdf, kind='linear')
    val = float( interp.__call__(percent))
    if invert :
        return 1-val
    return val

In [ ]:
interpolate_predicted_to_actual(actualCDF_values,predictedCDF_values,0.05,invert=False)

Likewise, we look at true events that fell above the 95% threshold, and check how many there are:

In [ ]:
interpolate_predicted_to_actual(actualCDF_values,predictedCDF_values,0.95,invert=True)